In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time

import os

# Data Capping

In [2]:
start_time = time.time()

directory_path = '../data/pems___101_N_Sep_2024/'
file_name_list = [f for f in os.listdir(directory_path) \
                  if os.path.isfile(os.path.join(directory_path, f)) and f[-1] == 'x' and f[0] != "~"]
file_name_list.sort()

thresh_lower = 0.1

# df = pd.read_excel(directory_path + file_name)
df_data_filtered = pd.read_csv(directory_path + 'data_filtered.csv', index_col=0)

dict_data_capped = {}
# dict_data_capped["Data Category"] = \
#     ["Station Type", "Number (data points)", "Mean (capped flow)", "Std Dev (capped flow)"]

times_dict = {}
for column_name_full in list(df_data_filtered.columns):
    print("column_name_full:", column_name_full)
    
    vals_list_str = df_data_filtered[column_name_full].tolist()
    
#     print("vals_list_str:", vals_list_str)

    column_name = column_name_full[:-5]
    
    dict_data_capped[column_name] = {}
    dict_data_capped[column_name]["Station type"] = vals_list_str[0]
    
    vals_list = [float(val) for val in vals_list_str[1:]]
    max_val = max(vals_list)
    dict_data_capped[column_name]["Flows (capped)"] = \
        [val for val in vals_list if val != 0.0 and val >= max_val * thresh_lower]
    
    
#     print()
#     print("vals_list:", vals_list)
#     print()
#     print("vals_array_capped:", vals_array_capped)
    
#     dict_data_capped[column_name] += [vals_array_capped.shape[0]]
#     dict_data_capped[column_name] += [np.mean(vals_array_capped)]
#     dict_data_capped[column_name] += [np.std(vals_array_capped)]

    
    
#     print("dict_data_capped[column_name]:", dict_data_capped[column_name])
#     print()


end_time = time.time()

print()
print("Time:", end_time - start_time)



column_name_full: 001___402376_Palo_Alto___main.xlsx
column_name_full: 002___402377_Palo_Alto___main.xlsx
column_name_full: 003___402379_Palo_Alto___main.xlsx
column_name_full: 004___425696_Palo_Alto___off.xlsx
column_name_full: 005___425697_Palo_Alto___main.xlsx
column_name_full: 006___425897_Palo_Alto___on.xlsx
column_name_full: 007___402380_Palo_Alto___main.xlsx
column_name_full: 008___404259_East_Palo_Alto___main.xlsx
column_name_full: 009___422116_East_Palo_Alto___main.xlsx
column_name_full: 010___403059_East_Palo_Alto___off.xlsx
column_name_full: 011___400859_East_Palo_Alto___main.xlsx
column_name_full: 012___403060_East_Palo_Alto___on.xlsx
column_name_full: 013___403061_East_Palo_Alto___on.xlsx
column_name_full: 014___404533_East_Palo_Alto___main.xlsx
column_name_full: 015___408267_East_Palo_Alto___on.xlsx
column_name_full: 016___400981_East_Palo_Alto___main.xlsx
column_name_full: 017___402398_East_Palo_Alto___main.xlsx
column_name_full: 018___404534_Menlo_Park___main.xlsx
colum

# Data Consolidation

In [3]:
# Assumption: The data starts and ends with main.
# Assumption: There is > 1 column.

column_name_list = list(dict_data_capped.keys())[1:]

dict_data_consolidated = {}

counter = 0
for column_index, column_name in enumerate(list(dict_data_capped.keys())):
#     print("column_index:", column_index)
    
    if column_index == 0:
        station_type_current = dict_data_capped[column_name]["Station type"]
        
        dict_data_consolidated[counter] = {}
        dict_data_consolidated[counter]["Station type"] = dict_data_capped[column_name]["Station type"]
        dict_data_consolidated[counter]["Start Station"] = column_name
        dict_data_consolidated[counter]["End Station"] = column_name
        dict_data_consolidated[counter]["Flow Data"] = dict_data_capped[column_name]["Flows (capped)"]
        
    elif column_index != 0 and dict_data_capped[column_name]["Station type"] != station_type_current:
        counter += 1
        
        station_type_current = dict_data_capped[column_name]["Station type"]
        
        dict_data_consolidated[counter] = {}
        dict_data_consolidated[counter]["Station type"] = dict_data_capped[column_name]["Station type"]
        dict_data_consolidated[counter]["Start Station"] = column_name
        dict_data_consolidated[counter]["End Station"] = column_name
        dict_data_consolidated[counter]["Flow Data"] = dict_data_capped[column_name]["Flows (capped)"]
        
    elif column_index != 0 and dict_data_capped[column_name]["Station type"] == station_type_current:
        dict_data_consolidated[counter]["End Station"] = column_name
        dict_data_consolidated[counter]["Flow Data"] += dict_data_capped[column_name]["Flows (capped)"]
        
    else:
        assert 1 == 0, "There should be no such case."
    


In [4]:
# dict_data_consolidated[0]["Flow Data"]

In [5]:
# Add (1) Number of data points, (2) Mean of capped flow, (3) Std Dev of capped flow, from flow data.

for counter in list(dict_data_consolidated.keys()):
    print("counter:", counter)
    print()
    
    flow_data_array = np.array(dict_data_consolidated[counter]["Flow Data"])
    print("flow_data_array:", flow_data_array)
    print()
    
    dict_data_consolidated[counter]["Number (data points)"] = flow_data_array.shape[0]
    dict_data_consolidated[counter]["Mean (capped flow)"] = np.mean(flow_data_array)
    dict_data_consolidated[counter]["Std Dev (capped flow)"] = np.std(flow_data_array)

dict_data_consolidated[0]

counter: 0

flow_data_array: [3258. 4324. 4520. 4025. 3506. 3300. 3159. 3331. 4228. 4318. 3837. 3518.
 3422. 3125. 3253. 4243. 4455. 4202. 3663. 3296. 3435. 3103. 4053. 3998.
 3775. 3342. 3287. 3325. 4039. 4944. 5117. 4760. 4195. 4008. 3966. 3391.
 4452. 4653. 4150. 3718. 3407. 3098. 3026. 3996. 4518. 4005. 3342. 3517.
 3229. 3800. 4706. 4612. 4146. 4000. 4024. 4065. 3886. 4681. 4786. 4430.
 3941. 3680. 3602. 3502. 4460. 4688. 4229. 3461. 3037. 3188. 3574. 4488.
 4606. 3984. 3608. 3216. 3158. 4143. 4289. 4595. 4272. 3658. 3419. 3163.
 3931. 4674. 4853. 4434. 4072. 4132. 4251. 3355. 4172. 4207. 3700. 3267.
 2980. 2713. 3446. 4273. 3892. 3947. 3062. 3461. 3067. 3406. 4241. 4119.
 3305. 3498. 3228. 3306. 3442. 4293. 4127. 3594. 3530. 3434. 3345. 3135.
 3919. 4099. 3394. 3246. 3590. 3155. 4161. 4714. 5055. 4600. 4316. 3942.
 3933. 4698. 5709. 6037. 5525. 4982. 4899. 4824. 4944. 5756. 5855. 5359.
 5019. 5017. 4783. 4860. 5720. 6025. 5828. 5232. 4944. 5092. 4606. 5556.
 5510. 5321. 4933. 499

{'Station type': 'main',
 'Start Station': '001___402376_Palo_Alto___main',
 'End Station': '003___402379_Palo_Alto___main',
 'Flow Data': [3258.0,
  4324.0,
  4520.0,
  4025.0,
  3506.0,
  3300.0,
  3159.0,
  3331.0,
  4228.0,
  4318.0,
  3837.0,
  3518.0,
  3422.0,
  3125.0,
  3253.0,
  4243.0,
  4455.0,
  4202.0,
  3663.0,
  3296.0,
  3435.0,
  3103.0,
  4053.0,
  3998.0,
  3775.0,
  3342.0,
  3287.0,
  3325.0,
  4039.0,
  4944.0,
  5117.0,
  4760.0,
  4195.0,
  4008.0,
  3966.0,
  3391.0,
  4452.0,
  4653.0,
  4150.0,
  3718.0,
  3407.0,
  3098.0,
  3026.0,
  3996.0,
  4518.0,
  4005.0,
  3342.0,
  3517.0,
  3229.0,
  3800.0,
  4706.0,
  4612.0,
  4146.0,
  4000.0,
  4024.0,
  4065.0,
  3886.0,
  4681.0,
  4786.0,
  4430.0,
  3941.0,
  3680.0,
  3602.0,
  3502.0,
  4460.0,
  4688.0,
  4229.0,
  3461.0,
  3037.0,
  3188.0,
  3574.0,
  4488.0,
  4606.0,
  3984.0,
  3608.0,
  3216.0,
  3158.0,
  4143.0,
  4289.0,
  4595.0,
  4272.0,
  3658.0,
  3419.0,
  3163.0,
  3931.0,
  4674.0,
  

# Data Consistency Enforcement

In [6]:
dict_data_consolidated_to_save = {}
dict_data_consolidated_to_save["Data Category"] \
    = ["Station Type", "Start Station", "End Station", \
       "Number (data points)", "Mean (capped flow)", "Std Dev (capped flow)"]

for counter in list(dict_data_consolidated.keys()):
    dict_data_consolidated_to_save[counter] = []
    dict_data_consolidated_to_save[counter] += [dict_data_consolidated[counter]["Station type"]]
    dict_data_consolidated_to_save[counter] += [dict_data_consolidated[counter]["Start Station"]]
    dict_data_consolidated_to_save[counter] += [dict_data_consolidated[counter]["End Station"]]
    dict_data_consolidated_to_save[counter] += [dict_data_consolidated[counter]["Number (data points)"]]
    dict_data_consolidated_to_save[counter] += [dict_data_consolidated[counter]["Mean (capped flow)"]]
    dict_data_consolidated_to_save[counter] += [dict_data_consolidated[counter]["Std Dev (capped flow)"]]

# df_dict_data_consolidated_to_save = pd.DataFrame(dict_data_consolidated_to_save)
# df_dict_data_consolidated_to_save.to_csv(directory_path + 'data_capped_consolidated.csv', index = False)


In [7]:
main_station_index_list = [counter for counter in list(dict_data_consolidated.keys()) \
                           if dict_data_consolidated[counter]["Station type"] == "main" ]

constraint_indices_list = []
for k in range(len(main_station_index_list) - 1):
    constraint_indices_list += [list(range(main_station_index_list[k], main_station_index_list[k+1]+1))]

print(constraint_indices_list)

num_constraints = len(constraint_indices_list)
num_variables = len(list(dict_data_consolidated.keys()))
constraint_matrix = np.zeros((num_constraints, num_variables))
# constraint_matrix.shape

for i in range(len(constraint_indices_list)):
    constraint_matrix[i, constraint_indices_list[i][0]] = 1
    for j in constraint_indices_list[i][1:-1]:
        if dict_data_consolidated[j]["Station type"] == "on":
            constraint_matrix[i, j] = 1
        elif dict_data_consolidated[j]["Station type"] == "off":
            constraint_matrix[i, j] = -1
        else:
            assert 1 == 0, "This case should not happen." 
    constraint_matrix[i, constraint_indices_list[i][-1]] = -1

constraint_matrix

[[0, 1, 2], [2, 3, 4], [4, 5, 6], [6, 7, 8], [8, 9, 10], [10, 11, 12], [12, 13, 14, 15], [15, 16, 17], [17, 18, 19], [19, 20, 21], [21, 22, 23, 24], [24, 25, 26], [26, 27, 28], [28, 29, 30], [30, 31, 32], [32, 33, 34], [34, 35, 36], [36, 37, 38], [38, 39, 40, 41], [41, 42, 43], [43, 44, 45], [45, 46, 47, 48], [48, 49, 50], [50, 51, 52], [52, 53, 54], [54, 55, 56], [56, 57, 58], [58, 59, 60]]


array([[ 1., -1., -1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1., -1., -1.]])

In [8]:
# print(np.fill_diagonal(np.zeros((3, 3)), [1, 2, 3]))
# print(np.diag([1, 2, 3]))

In [9]:
# Variables:    
data_var = cp.Variable(num_variables)

# Cost Function:
func = 0.0
data_mean = np.array([dict_data_consolidated[j]["Mean (capped flow)"] for j in range(num_variables)])
kernel = np.diag([1/dict_data_consolidated[j]["Std Dev (capped flow)"]**2 \
                  for j in range(num_variables)])
func += cp.quad_form(data_var - data_mean, kernel)

# Objective:
objective = cp.Minimize(func)

# Constraints:
constraints = []
constraints += [constraint_matrix @ data_var == 0.0]
constraints += [data_var >= 0.0]

# Problem:
prob = cp.Problem(objective, constraints)

# Solve:
result = prob.solve()

# Extract Values:

data_var_value = data_var.value
print("data_var_value.shape:", data_var_value.shape)
print("data_var_value:", data_var_value)


data_var_value.shape: (61,)
data_var_value: [5494.68905277  601.70895548 4892.98010129  829.57333565 5722.55344342
   90.96836237 5631.58509401  312.39972752 5943.98481905  237.64816387
 6181.63295003  819.69959588 5361.93323126  714.91493699  117.10181871
 4764.11970177   57.43373443 4821.5533115   184.3358917  5005.88907949
  136.51959217 5142.40824282  429.25129758  490.1337696  5203.29061466
  246.80559562 5450.09618676  907.11600843 4542.98016586  450.84599836
 4993.82615365 1023.29593038 6017.12207286  191.78901691 6208.91108095
  512.8205555  5696.0905206   361.50448548 6057.59500584  577.82564538
  430.37127341 5910.14063596   80.48339332 5829.65724588   88.80086555
 5918.45811268  723.17162929  562.28811846 5757.57460087  140.87813378
 5898.4527308   590.14292859 6488.59565312 1134.96814523 5353.62750433
  485.74973193 5839.37723685  577.52036294 5261.85687843 1145.1320318
 4116.72484925]


In [12]:
kernel.shape

(61, 61)

In [10]:
dict_data_consolidated_to_save["Data Category"] += ["Consistent Flow"]

for counter in list(dict_data_consolidated.keys()):
    print("data_var_value[counter]:", data_var_value[counter])
    dict_data_consolidated_to_save[counter].append(data_var_value[counter])

print()
print("dict_data_consolidated_to_save[10]:", dict_data_consolidated_to_save[10])
print()
print("dict_data_consolidated_to_save:", dict_data_consolidated_to_save)

df_dict_data_consolidated_to_save = pd.DataFrame(dict_data_consolidated_to_save)
df_dict_data_consolidated_to_save.to_csv(directory_path + 'data_capped_consolidated.csv', index = False)


data_var_value[counter]: 5494.689052769372
data_var_value[counter]: 601.7089554781742
data_var_value[counter]: 4892.980101290639
data_var_value[counter]: 829.5733356500147
data_var_value[counter]: 5722.553443420686
data_var_value[counter]: 90.96836237495555
data_var_value[counter]: 5631.585094006992
data_var_value[counter]: 312.3997275221162
data_var_value[counter]: 5943.984819052002
data_var_value[counter]: 237.64816387341074
data_var_value[counter]: 6181.632950031043
data_var_value[counter]: 819.6995958765693
data_var_value[counter]: 5361.933231258203
data_var_value[counter]: 714.9149369937513
data_var_value[counter]: 117.10181870599753
data_var_value[counter]: 4764.119701771828
data_var_value[counter]: 57.43373442705964
data_var_value[counter]: 4821.553311501502
data_var_value[counter]: 184.33589170364948
data_var_value[counter]: 5005.889079493303
data_var_value[counter]: 136.5195921737947
data_var_value[counter]: 5142.4082428231195
data_var_value[counter]: 429.2512975776031
data_va

In [ ]:
# constraint_signs_list = []
# for k in range(len(main_station_index_list) - 1):
#     signs_list = []
#     signs_list += [1]
#     for index in constraint_indices_list[k][1:-1]:
#         if dict_data_consolidated[index]["Station type"] == "on":
#             signs_list += [1]
#         elif dict_data_consolidated[index]["Station type"] == "off":
#             signs_list += [-1]
#         else:
#             assert 1 == 0, "This case should not happen."
#     signs_list += [-1]
#     constraint_signs_list += [signs_list]

# print()
# print(constraint_signs_list)

# Scratch Work

In [ ]:
ell = [1, 2, 3, 4, 5]
# ell[1:-1]
ell += [6]
ell